<a href="https://colab.research.google.com/github/AMIRMOHAMMAD-OSS/Bioinformatics/blob/main/Transformer_Xylanase.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.0/281.0 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 79.6 MB/s eta 0:00:00


In [ ]:
from Bio import SeqIO
import torch

In [ ]:
GH11 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH11_protein_sequences.fasta", "fasta")]))
GH10 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH10_protein_sequences.fasta", "fasta")]))
GH12 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH12_protein_sequences.fasta", "fasta")]))
GH9 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH9_protein_sequences.fasta", "fasta")]))
GH8 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH8_protein_sequences.fasta", "fasta")]))

In [ ]:
pos = torch.concat((Encode(Padding([edit(i) for i in GH10 if len(i)<= 512])),torch.ones((5356,1))),1)
neg = torch.concat((torch.concat((Encode(Padding([edit(i) for i in GH11 if len(i)<= 512])),Encode(Padding([edit(i) for i in GH12 if len(i)<= 512])),Encode(Padding([edit(i) for i in GH9 if len(i)<= 512])),Encode(Padding([edit(i) for i in GH8 if len(i)<= 512])))),torch.zeros((9346,1))),1)
Dataset = torch.concat((pos,neg)).to(torch.long).to("cuda")
from sklearn.model_selection import train_test_split as TTS
train_dataset,test_dataset = TTS(Dataset,random_state = 48 , test_size = 0.05)

In [ ]:
Dataset = torch.concat((pos,neg)).to(torch.long).to("cuda")

In [ ]:
from sklearn.model_selection import train_test_split as TTS
train_dataset,test_dataset = TTS(Dataset,random_state = 48 , test_size = 0.05)

torch.Size([736, 513])

In [ ]:
import torch
def Padding(x,PAD = 0,max_len = 512):
  l = []
  max_len = 512
  for i in x:
    a = i
    if len(a) < max_len:
      a = a + "X"*(max_len-len(a))
    l.append(a)
  return l
def edit(x):
    for i in x:
      if i in "BJ\tOUX\Z_nabc\ndefNghijklmnopqrstuvwxyz":
        x = x.replace(i,"")
    return x
def Encode(seqs):
    cha = "XACDEFGHIKLMNPQRSTVWY"
    return torch.vstack(tuple(torch.tensor([cha.index(i) for i in seq]) for seq in seqs)).to(torch.long).view(len(seqs), len(seqs[0]))

In [ ]:
from tqdm import trange,tqdm
import numpy as np
from tqdm import trange,tqdm
import numpy as np
import torch
from Bio import SeqIO
import torch
GH11 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH11_protein_sequences.fasta", "fasta")]))
GH10 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH10_protein_sequences.fasta", "fasta")]))
GH12 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH12_protein_sequences.fasta", "fasta")]))
GH9 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH9_protein_sequences.fasta", "fasta")]))
GH8 = list(set([str(record.seq) for record in SeqIO.parse("/content/GH8_protein_sequences.fasta", "fasta")]))
def Padding(x,PAD = 0,max_len = 512):
  l = []
  max_len = 512
  for i in x:
    a = i
    if len(a) < max_len:
      a = a + "X"*(max_len-len(a))
    l.append(a)
  return l
def edit(x):
    for i in x:
      if i in "BJ\tOUX\Z_nabc\ndefNghijklmnopqrstuvwxyz":
        x = x.replace(i,"")
    return x
def Encode(seqs):
    cha = "XACDEFGHIKLMNPQRSTVWY"
    return torch.vstack(tuple(torch.tensor([cha.index(i) for i in seq]) for seq in seqs)).to(torch.long).view(len(seqs), len(seqs[0]))
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

def setup_logging(config):
    work_dir = config.system.work_dir
    os.makedirs(work_dir, exist_ok=True)
    with open(os.path.join(work_dir, 'args.txt'), 'w') as f:
        f.write(' '.join(sys.argv))
    with open(os.path.join(work_dir, 'config.json'), 'w') as f:
        f.write(json.dumps(config.to_dict(), indent=4))

class CfgNode:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)

    def __str__(self):
        return self._str_helper(0)

    def _str_helper(self, indent):
        parts = []
        for k, v in self.__dict__.items():
            if isinstance(v, CfgNode):
                parts.append("%s:\n" % k)
                parts.append(v._str_helper(indent + 1))
            else:
                parts.append("%s: %s\n" % (k, v))
        parts = [' ' * (indent * 4) + p for p in parts]
        return "".join(parts)

    def to_dict(self):
        return { k: v.to_dict() if isinstance(v, CfgNode) else v for k, v in self.__dict__.items() }

    def merge_from_dict(self, d):
        self.__dict__.update(d)

    def merge_from_args(self, args):
        for arg in args:
            keyval = arg.split('=')
            assert len(keyval) == 2, "expecting each override arg to be of form --arg=value, got %s" % arg
            key, val = keyval
            try:
                val = literal_eval(val)
            except ValueError:
                pass

            assert key[:2] == '--'
            key = key[2:]
            keys = key.split('.')
            obj = self
            for k in keys[:-1]:
                obj = getattr(obj, k)
            leaf_key = keys[-1]
            assert hasattr(obj, leaf_key), f"{key} is not an attribute that exists in the config"
            print("command line overwriting config attribute %s with %s" % (key, val))
            setattr(obj, leaf_key, val)
import math
import torch
import torch.nn as nn
from torch.nn import functional as F

class SiLU(nn.Module):
   def forward(self, x):
        return x*F.sigmoid(x)

class NY(nn.Module):
  def forward(self,x):
    return 3*torch.tanh(0.3*x)

class NewGELU(nn.Module):
    def forward(self, x):
        return 0.5 * x * (1.0 + torch.tanh(math.sqrt(2.0 / math.pi) * (x + 0.044715 * torch.pow(x, 3.0))))

class SelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd)
        self.attn_dropout = nn.Dropout(config.attn_pdrop)
        self.resid_dropout = nn.Dropout(config.resid_pdrop)
        self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                     .view(1, 1, config.block_size, config.block_size))
        self.n_head = config.n_head
        self.n_embd = config.n_embd

    def forward(self, x):
        B, T, C = x.size()
        q, k ,v  = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
        att = att.masked_fill(self.bias[:,:,:T,:T] == 0, float('-inf'))
        att = F.softmax(att, dim=-1)
        att = self.attn_dropout(att)
        y = att @ v
        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = nn.LayerNorm(config.n_embd)
        self.attn = nn.MultiheadAttention(config.n_embd, config.n_head, dropout=config.attn_pdrop)
        self.ln2 = nn.LayerNorm(config.n_embd)
        self.mlp = nn.Sequential(
            nn.Linear(config.n_embd, 4 * config.n_embd),
            nn.GELU(),
            nn.Linear(4 * config.n_embd, config.n_embd),
            nn.Dropout(config.resid_pdrop),
        )

    def forward(self, x, attention_mask=None):
        if attention_mask is not None:
            attention_mask = attention_mask.transpose(0, 1)

        x = x + self.attn(self.ln1(x), self.ln1(x), self.ln1(x), key_padding_mask=attention_mask)[0]
        x = x + self.mlp(self.ln2(x))
        return x



class ClassifierI(nn.Module):
    @staticmethod
    def get_default_config():
        C = CfgNode()
        C.model_type = 'gpt'
        C.n_layer = None
        C.n_head = None
        C.n_embd =  None
        C.vocab_size = 21
        C.max_length = 512
        C.embd_pdrop = 0.1
        C.resid_pdrop = 0.1
        C.attn_pdrop = 0.1
        return C

    def __init__(self, config):
        super().__init__()
        self.max_length = config.block_size
        self.soft = nn.Softmax(1)
        self.config = self.get_default_config()
        self.device = "cuda"
        self.model_states = {
            'h': {'n_layer': 48, 'n_head': 25, 'n_embd': 1600},
            'g': {'n_layer': 12, 'n_head': 12, 'n_embd': 768},
            'f': {'n_layer': 24, 'n_head': 16, 'n_embd': 1024},
            'e': {'n_layer': 36, 'n_head': 20, 'n_embd': 1280},
            'd': {'n_layer': 8, 'n_head': 16, 'n_embd': 512},
            'c': {'n_layer': 6, 'n_head': 6, 'n_embd': 192},
            'b': {'n_layer': 4, 'n_head': 4, 'n_embd': 128},
            'a': {'n_layer': 3, 'n_head': 3, 'n_embd': 48}
        }

        type_ = config.model_type is not None
        p = all([config.n_layer is not None, config.n_head is not None, config.n_embd is not None])
        if type_:
            config.merge_from_dict(self.model_states[config.model_type])

        self.loss_fn = nn.BCEWithLogitsLoss()
        self.ny = NY()
        self.soft = nn.Softmax()
        self.transformer = nn.ModuleDict(dict(
            wte = nn.Embedding(config.vocab_size, config.n_embd),
            wpe = nn.Embedding(config.max_length, config.n_embd),
            drop = nn.Dropout(config.embd_pdrop),
            h = nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f = nn.LayerNorm(config.n_embd),
        ))
        self.classifier_head = nn.Sequential(
            nn.Linear(config.n_embd, 2)
        )
        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                torch.nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

        n_params = sum(p.numel() for p in self.transformer.parameters())
        print("[ Number of trainable parameters: %.2fM ]" % (n_params / 1e6,))

    def forward(self, idx, attention_mask=None, targets=None):
        device = idx.device
        b, t = idx.size()
        pos = torch.arange(0, t, dtype=torch.long, device=device).unsqueeze(0)
        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)
        x = self.transformer.drop(tok_emb + pos_emb)
        for block in self.transformer.h:
            x = block(x, attention_mask=attention_mask)
        x = self.transformer.ln_f(x)
        logits = self.soft(self.classifier_head(x).mean(1).to(device))
        loss = None
        if targets is not None:
            loss = self.loss_fn(logits, targets)
        return logits, loss

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
        elif isinstance(module, nn.LayerNorm):
            torch.nn.init.zeros_(module.bias)
            torch.nn.init.ones_(module.weight)

    @classmethod
    def from_pretrained(cls, model_type):
        from transformers import GPT2LMHeadModel
        config = cls.get_default_config()
        config.model_type = model_type
        config.vocab_size = 25
        config.max_length = 512
        model = ClassifierI(config)
        sd = model.state_dict()
        model_hf = GPT2LMHeadModel.from_pretrained(model_type)
        sd_hf = model_hf.state_dict()
        keys = [k for k in sd_hf if not k.endswith('attn.masked_bias')]
        transposed = ['attn.c_attn.weight', 'attn.c_proj.weight', 'mlp.c_fc.weight', 'mlp.c_proj.weight']
        assert len(keys) == len(sd)
        for k in keys:
            if any(k.endswith(w) for w in transposed):
                assert sd_hf[k].shape[::-1] == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k].t())
            else:
                assert sd_hf[k].shape == sd[k].shape
                with torch.no_grad():
                    sd[k].copy_(sd_hf[k])
        return model

    def configure_optimizers(self, train_config):
        decay = set()
        no_decay = set()
        whitelist_weight_modules = (torch.nn.Linear, )
        blacklist_weight_modules = (torch.nn.LayerNorm, torch.nn.Embedding)
        for mn, m in self.named_modules():
            for pn, p in m.named_parameters():
                fpn = '%s.%s' % (mn, pn) if mn else pn
                if pn.endswith('bias'):
                    no_decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, whitelist_weight_modules):
                    decay.add(fpn)
                elif pn.endswith('weight') and isinstance(m, blacklist_weight_modules):
                    no_decay.add(fpn)
        param_dict = {pn: p for pn, p in self.named_parameters()}
        inter_params = decay & no_decay
        union_params = decay | no_decay
        optim_groups = [
            {"params": [param_dict[pn] for pn in sorted(list(decay))], "weight_decay": 0},
            {"params": [param_dict[pn] for pn in sorted(list(no_decay))], "weight_decay": 0.0},]
        optimizer = torch.optim.AdamW(optim_groups, lr=0.0008)
        return optimizer

    @torch.no_grad
    def predict_proba(self,idx,attention_mask,j ="None"):
      sigmoid = nn.Sigmoid()
      soft = nn.Softmax(dim=1)
      ny = NY()
      si = SiLU()
      self.eval()
      l,n = idx.shape
      x,_ = self(idx,attention_mask)
      if j == "None":
        return x[:,1]
      elif j == "soft":
        return self.soft(x)[:,0:1]
      else:
        return sigmoid(x)[:,0:1]

    @torch.no_grad
    def predict(self,idx,attention_mask):
      y = self.predict_proba(idx,attention_mask,"None").view(1,idx.shape[0]).tolist()[0]
      y_pred = [i>=0.5 for i in y]
      return y_pred,


from sklearn.model_selection import KFold
import numpy as np
import torch
from sklearn.metrics import accuracy_score

class Trainer:
    @staticmethod
    def get_default_config():
        C = CfgNode()
        C.device = 'cuda'
        C.num_workers = 4
        C.max_iters = None
        C.batch_size = 64
        C.max_length = 512
        C.learning_rate = 0.001
        C.betas = (0.9, 0.95)
        C.weight_decay = 0.1
        C.grad_norm_clip = 1.0
        C.patience = 3
        return C

    def __init__(self, config, model, train_data, val_data):
        self.config = config
        self.model = model
        self.optimizer = None
        self.train_data = train_data
        self.val_data = val_data
        self.device = "cuda"
        self.model = self.model.to(self.device)
        self.iter_num = 0
        self.iter_time = 0.0
        self.iter_dt = 0.0
        self.best_val_loss = float('inf')
        self.early_stop_counter = 0

    def get_batch(self, data, batch_size):
        indices = np.random.permutation(len(data))[:batch_size]
        batch = data[indices]
        seq = batch[:, :-1]
        targets = batch[:, -1]
        targets = torch.nn.functional.one_hot(torch.tensor(targets, dtype=torch.int64), num_classes=2).float().to(self.device)
        seq = torch.tensor(seq, dtype=torch.long).to(self.device)
        attention_masks = (seq != 0).float().to(self.device).view(batch_size, 512)
        return seq, attention_masks, targets

    def run(self):
        self.optimizer = self.model.configure_optimizers(self.config)
        batch_size = 64

        for epoch in range(50):
            self.model.train()
            for _ in trange(100):
                seq, attention_mask, targets = self.get_batch(self.train_data, batch_size)
                logits, loss = self.model(seq, attention_mask, targets)
                self.optimizer.zero_grad()
                loss.backward()
                torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.config.grad_norm_clip)
                self.optimizer.step()

            train_loss, val_loss = self.evaluate()
            print(f"\n[train loss = {train_loss}, val loss =  {val_loss}]\n")

            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.early_stop_counter = 0
            else:
                self.early_stop_counter += 1
                if self.early_stop_counter >= self.config.patience:
                    print(f"Early stopping at epoch {epoch}")
                    break

            # Validate and print accuracy
            test_preds, test_labels = self.predict_and_evaluate()
            acc = accuracy_score(test_labels, test_preds)
            print(f"Validation accuracy = {acc}")

        return self.model, test_preds

    def evaluate(self):
        self.model.eval()
        losses = {'train': [], 'val': []}
        for mode in ['train', 'val']:
            dataset = self.train_data if mode == 'train' else self.val_data
            for _ in range(200):
                seq, attention_mask, targets = self.get_batch(dataset, self.config.batch_size)
                logits, loss = self.model(seq, attention_mask, targets)
                losses[mode].append(loss.item())
        self.model.train()
        return np.mean(losses['train']), np.mean(losses['val'])

    def predict_and_evaluate(self):
        self.model.eval()
        idx = self.val_data[:, :-1].to(self.device)
        attention_masks = (idx != 0).float().to(self.device).view(idx.shape[0], 512)
        test_preds = self.model.predict(idx, attention_masks)[0]
        test_labels = self.val_data[:, -1].tolist()
        return test_preds, test_labels

import matplotlib.pyplot as plt

def cross_validate(model_class, data, labels, config):
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    accuracies = []

    for fold, (train_index, val_index) in enumerate(kf.split(data), 1):
        train_data, val_data = data[train_index], data[val_index]
        train_labels, val_labels = labels[train_index], labels[val_index]

        model = model_class(config)
        trainer = Trainer(config, model, train_data, val_data)
        trained_model, test_preds = trainer.run()

        acc = accuracy_score(val_labels, test_preds)
        accuracies.append(acc)
        print(f'Fold {fold} Accuracy: {acc:.4f}')

    print(f'Average Accuracy: {np.mean(accuracies):.4f}')
    return accuracies


model_config = ClassifierI.get_default_config()
model_config.model_type = 'b'
model_config.vocab_size = 21
model_config.block_size = 512

model = ClassifierI(model_config)
train_config = Trainer.get_default_config()
train_config.learning_rate = 0.0008
train_config.max_iters = 2000
train_config.num_workers = 0
train_config.patience = 5
trainer = Trainer(train_config, model, train_dataset, test_dataset)

